In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_table("Restaurant_Reviews.tsv")

In [ ]:
df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.,1
4,The selection on the menu was great and so were the prices.,1


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# Data Preparation

In [ ]:
!pip install contractions
import contractions
import nltk
import re
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

def fix_contractions(text):
    expanded_words = []   
    for word in text.split():
        expanded_words.append(contractions.fix(word))  
   
    return ' '.join(expanded_words)

def clean(text):
    return re.sub(r"\s+", " ", re.sub(r"[^\sA-Za-z0-9]", "", re.sub(r'[^\w\s]', ' ', text))).lower()
    # return " ".join(map(spellchecker.correction, re.sub(r"\s+", " ", text).lower().split()))

def tokenize(text):
    return word_tokenize(text)

STOP_WORDS = set(stopwords.words("english"))
STOP_WORDS.remove("not")
STOP_WORDS.remove("no")

def filter_stopwords(tokenized_text):
    return list(filter(lambda x: x not in STOP_WORDS, tokenized_text))

def lemmatize(filtered_text):
    return list(map(WordNetLemmatizer().lemmatize, filtered_text))

# def lemmatize_custom(sentence):
#     return " ".join(list(map(WordNetLemmatizer().lemmatize, filter_stopword(word_tokenize(sentence)))))

def stemming(lemmatized_text):
    return " ".join(list(map(SnowballStemmer("english").stem, lemmatized_text)))

def preprocess(text):
    pipeline = [fix_contractions, clean, tokenize, filter_stopwords, lemmatize]
    for process in pipeline:
        text = process(text)

    return " ".join(text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 32.3 MB/s eta 0:00:00


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
df["cleaned_review"] = df["Review"].apply(preprocess)

In [ ]:
positive_df = df[df["Liked"] == 1]
negative_df = df[df["Liked"] == 0]